# Leash Tutorial - ECFPs and Random Forest
## Introduction

There are many ways to represent molecules for machine learning. 

In this tutorial we will go through one of the simplest: ECFPs [[1]](https://pubs.acs.org/doi/10.1021/ci100050t) and Random Forest. This technique is surprisingly powerful, and on previous benchmarks often gets uncomfortably close to the state of the art.

First molecule graphs are broken into bags of subgraphs of varying sizes.

![ecfp featurizing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_generation.png)

Then the bag of subgraphs is hashed into a bit vector

![hashing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_folding.png)

This can be thought of as analogous to the [hashing trick](https://en.wikipedia.org/wiki/Feature_hashing) [[2]](https://alex.smola.org/papers/2009/Weinbergeretal09.pdf) on bag of words for NLP problems, from the days before transformers. 

RDKit, an open-source cheminformatics tool, is used for generating ECFP features. It facilitates the creation of hashed bit vectors, streamlining the process. We can install it as follows:

In [1]:
import rdkit

The training set is pretty big, but we can treat the parquet files as databases using duckdb. We will use this to sample down to a smaller dataset for demonstration purposes. Lets install duckdb as well.

In [2]:
import duckdb

## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import os
print(os.getcwd())

/home/louis/Documents/personal_coding/graph_neural_networks/BELKA_project


In [4]:
import duckdb
import pandas as pd

train_path = 'data/train.parquet'
test_path = 'data/test.parquet'


con = duckdb.connect()

df_temp = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 5000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 5000)""").df()

con.close()
           

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder



# Convert SMILES to RDKit molecules
df_temp['molecule'] = df_temp['molecule_smiles'].apply(Chem.MolFromSmiles)

"""
# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))
"""
from rdkit.Chem import rdFingerprintGenerator

mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)

df_temp['ecfp'] = df_temp['molecule'].apply(mfpgen.GetFingerprint).apply(lambda bitvec: list(bitvec))

# df_temp['ecfp'] = df_temp['molecule'].apply(generate_ecfp)



In [6]:
df_temp.to_pickle("data/ecfp.pkl")

In [7]:
df = pd.read_pickle("data/ecfp.pkl")

In [34]:
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,221019501,O=C(Nc1cncc(C(=O)O)n1)OCC1c2ccccc2-c2ccccc21,Nc1ccc([N+](=O)[O-])c(F)c1,Nc1nc2c(s1)CCCC2,O=C(N[Dy])c1cncc(Nc2nc(Nc3ccc([N+](=O)[O-])c(F...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7e220b848f90>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,191330190,O=C(Nc1ccc(C(=O)O)cc1F)OCC1c2ccccc2-c2ccccc21,Cl.NCc1nnc(-c2ccncc2)[nH]1,Cl.Cl.NCC1(N2CCOCC2)CCOCC1,O=C(N[Dy])c1ccc(Nc2nc(NCc3nnc(-c4ccncc4)[nH]3)...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7e220b848f40>,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,72980228,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ncccc1F,Cl.NCCCOc1cccc(F)c1,O=C(N[Dy])C(CC1CCCCC1)Nc1nc(NCCCOc2cccc(F)c2)n...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7e220b5c9040>,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,95444605,O=C(N[C@@H](Cc1ccc(Cl)c(Cl)c1)C(=O)O)OCC1c2ccc...,Cl.NCc1nc2ccccc2c(=O)[nH]1,Nc1nc2c(s1)CCCC2,O=C(N[Dy])[C@H](Cc1ccc(Cl)c(Cl)c1)Nc1nc(NCc2nc...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7e220b5c9090>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,291614515,O=C(O)[C@H]1COCCN1C(=O)OCC1c2ccccc2-c2ccccc21,NC1=NC(=O)CS1,NCCN1CC2CCC1C2,O=C1CSC(Nc2nc(NCCN3CC4CCC3C4)nc(N3CCOC[C@@H]3C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7e220b5c90e0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
9995,151942126,O=C(Nc1c(F)cccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1cncc(F)c1,NCC(O)COc1cccc(Cl)c1Cl,O=C(N[Dy])c1cccc(F)c1Nc1nc(NCc2cncc(F)c2)nc(NC...,HSA,1,<rdkit.Chem.rdchem.Mol object at 0x7e220b29cd10>,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9996,74448765,O=C(NC(Cc1ccccc1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Cc1cc(N)ccc1F,CCn1nccc1CN,CCn1nccc1CNc1nc(Nc2ccc(F)c(C)c2)nc(NC(Cc2ccccc...,BRD4,1,<rdkit.Chem.rdchem.Mol object at 0x7e220b29cd60>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9997,72782997,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],Cc1cnc(Cl)cc1N,Cc1cnc(Cl)cc1Nc1nc(Nc2cc(N3CCNCC3)ccc2[N+](=O)...,BRD4,1,<rdkit.Chem.rdchem.Mol object at 0x7e220b29cdb0>,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9998,280026013,O=C(O)[C@@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,CC(C)(C)OC(=O)N1Cc2c(N)n[nH]c2C1(C)C,Nc1nc2nc[nH]c2c(=O)[nH]1,CC(C)(C)OC(=O)N1Cc2c(Nc3nc(Nc4nc5nc[nH]c5c(=O)...,HSA,1,<rdkit.Chem.rdchem.Mol object at 0x7e220b29ce00>,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Train Model

In [9]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.94


In [44]:
protein_onehot

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [43]:
protein_onehot.shape

(10000, 3)

In [48]:
df['ecfp']

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
3       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
9995    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9996    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9997    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9998    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9999    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: ecfp, Length: 10000, dtype: object

In [41]:
len(df['ecfp'].tolist()), len(protein_onehot.tolist())

(10000, 10000)

In [36]:
x_temp = list(zip(df['ecfp'].tolist(), protein_onehot.tolist()))
x_temp[0]

2

In [38]:
len(y)

10000

In [33]:
len(X[0])

2051

In [22]:
[1,2,3]+[4,5,6]

[1, 2, 3, 4, 5, 6]

In [10]:
len(X_train[0])

2051

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [17]:
import os

# Process the test.parquet file chunk by chunk
test_file = 'data/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):
    print("gone through one loop")

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(mfpgen.GetFingerprint).apply(lambda bitvec: list(bitvec))

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))


gone through one loop


KeyboardInterrupt: 